<a href="https://colab.research.google.com/github/jamelof23/ASL/blob/main/InterFaceGAN-Jam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fetch Codebase and Models

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'interfacegan'
!git clone https://github.com/genforce/interfacegan.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')
!wget https://www.dropbox.com/s/t74z87pk3cf8ny7/pggan_celebahq.pth?dl=1 -O models/pretrain/pggan_celebahq.pth --quiet
!wget https://www.dropbox.com/s/nmo2g3u0qt7x70m/stylegan_celebahq.pth?dl=1 -O models/pretrain/stylegan_celebahq.pth --quiet
!wget https://www.dropbox.com/s/qyv37eaobnow7fu/stylegan_ffhq.pth?dl=1 -O models/pretrain/stylegan_ffhq.pth --quiet

# Define Utility Functions

In [ ]:
import os.path
import io
import IPython.display
import numpy as np
import cv2
import PIL.Image

import torch

# A dictionary containing information about the models available (e.g., pggan, stylegan).
from models.model_settings import MODEL_POOL

from models.pggan_generator import PGGANGenerator
from models.stylegan_generator import StyleGANGenerator
from utils.manipulator import linear_interpolate


def build_generator(model_name):
  """Builds the generator by model name."""
  gan_type = MODEL_POOL[model_name]['gan_type']
  if gan_type == 'pggan':
    generator = PGGANGenerator(model_name)
  elif gan_type == 'stylegan':
    generator = StyleGANGenerator(model_name)
  return generator


def sample_codes(generator, num, latent_space_type='Z', seed=0):
  """Samples latent codes randomly."""
  #Setting a random seed ensures reproducibility. Every time the function is called with the same seed, it generates the same latent codes are typically sampled from a standard normal distribution.
  np.random.seed(seed)
  #The function uses generator.easy_sample(num) to sample num latent vectors from the Z space.
  codes = generator.easy_sample(num)
  print(f"[INFO] Latent codes in Z space (before mapping):")
  print(f"Shape: {codes.shape}")
  print(f"Sample data: {codes[0, :5]}")  # Print first 5 elements of the first vector
  #This condition checks if the generator is using StyleGAN and whether we want to use the W space.
  if generator.gan_type == 'stylegan' and latent_space_type == 'W':
    #he codes sampled in Z space are first converted to a PyTorch tensor and moved to the device (CPU or GPU):
    codes = torch.from_numpy(codes).type(torch.FloatTensor).to(generator.run_device)
    # Print information before mapping
    print(f"[INFO] Latent codes as tensor (before mapping to W space):")
    print(f"Shape: {codes.shape}")
    print(f"Sample data: {codes[0, :5]}") #extracts the first 5 elements of the first latent vector.
    #The codes are then passed through the StyleGAN's mapping network to convert them into the W space:
    codes = generator.get_value(generator.model.mapping(codes))
    # Print information after mapping to W space
    print(f"[INFO] Latent codes in W space (after mapping):")
    print(f"Shape: {codes.shape}")
    print(f"Sample data: {codes[0, :5]}") #extracts the first 5 elements of the first latent vector.
  #Finally, the function returns the latent codes, which can either be in the Z space or transformed to the W space, depending on the inputs.
  #The mapping network transforms Z vectors into the W space, which is disentangled.
  return codes


#main function display a grid of images

#images: A NumPy array containing multiple images. The expected shape is (num, height, width, channels), where:
#col: The number of columns to display in the grid.
#viz_size: The size (in pixels) to which each image will be resized before displaying (default is 256).
def imshow(images, col, viz_size=256):
  """Shows images in one figure."""
  # Extracting the Shape of the Images
  num, height, width, channels = images.shape
  #Ensures that the number of images (num) is divisible by the number of columns (col).
  #This makes sure that the images can be arranged into a complete grid without leaving any empty cells.
  assert num % col == 0
  #Computes the number of rows needed to fit all the images in the grid.
  row = num // col

  #Creates an empty canvas (a blank image) to hold all the images in a grid format.
  #The canvas has dimensions (viz_size * row, viz_size * col, channels), where:
  #viz_size * row: Total height of the grid.
  #viz_size * col: Total width of the grid.
  #channels: Number of color channels (same as the input images).
  fused_image = np.zeros((viz_size * row, viz_size * col, channels), dtype=np.uint8)

  #Placing Each Image on the Canvas
  for idx, image in enumerate(images):
    i, j = divmod(idx, col)
    y = i * viz_size
    x = j * viz_size
    #Resizing and Copying Each Image to the Canvas
    if height != viz_size or width != viz_size:
      image = cv2.resize(image, (viz_size, viz_size))
    fused_image[y:y + viz_size, x:x + viz_size] = image

  #Converting the Canvas to a Displayable Format
  fused_image = np.asarray(fused_image, dtype=np.uint8)
  data = io.BytesIO()
  PIL.Image.fromarray(fused_image).save(data, 'jpeg')
  im_data = data.getvalue()
  #Displaying the Image Using IPython
  disp = IPython.display.display(IPython.display.Image(im_data))
  return disp

# Select a Model

In [ ]:
#@title { display-mode: "form", run: "auto" }
model_name = "stylegan_ffhq" #@param ['pggan_celebahq','stylegan_celebahq', 'stylegan_ffhq']
latent_space_type = "W" #@param ['Z', 'W']

generator = build_generator(model_name)

ATTRS = ['age', 'eyeglasses', 'gender', 'pose', 'smile']
boundaries = {}
for i, attr_name in enumerate(ATTRS):
  boundary_name = f'{model_name}_{attr_name}'
  if generator.gan_type == 'stylegan' and latent_space_type == 'W':
    boundaries[attr_name] = np.load(f'boundaries/{boundary_name}_w_boundary.npy')
  else:
    boundaries[attr_name] = np.load(f'boundaries/{boundary_name}_boundary.npy')

# Sample latent codes

In [ ]:
#@title { display-mode: "form", run: "auto" }

num_samples = 4 #@param {type:"slider", min:1, max:8, step:1}
noise_seed = 0 #@param {type:"slider", min:0, max:1000, step:1}

latent_codes = sample_codes(generator, num_samples, latent_space_type, noise_seed)
if generator.gan_type == 'stylegan' and latent_space_type == 'W':
  synthesis_kwargs = {'latent_space_type': 'W'}
else:
  synthesis_kwargs = {}

images = generator.easy_synthesize(latent_codes, **synthesis_kwargs)['image']
imshow(images, col=num_samples)

# Edit facial attributes

In [ ]:
#@title { display-mode: "form", run: "auto" }

age = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
eyeglasses = 0 #@param {type:"slider", min:-2.9, max:3.0, step:0.1}
gender = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
pose = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
smile = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}

new_codes = latent_codes.copy()
for i, attr_name in enumerate(ATTRS):
  new_codes += boundaries[attr_name] * eval(attr_name)

new_images = generator.easy_synthesize(new_codes, **synthesis_kwargs)['image']
imshow(new_images, col=num_samples)